Sample use of the algorithm with Jupyter.

# Notebook initialization
## Download module

In [ ]:
import os
import urllib.request

if not os.path.isfile('mtf_estimator_algorithm.py'):
    urllib.request.urlretrieve('https://raw.githubusercontent.com/JorgeGIlG/MTF_Estimator/devel/qgis_plugin/mtf_estimator/mtf_estimator_algorithm.py',
                               'mtf_estimator_algorithm.py')


## Download sample image

In [ ]:
if not os.path.isfile('baotou_CALVAL_L0R_000000_20200328T032332_202003T032334_MTF_12196_7333.tif'):
    urllib.request.urlretrieve('https://github.com/JorgeGIlG/MTF_Estimator/raw/devel/data/baotou_CALVAL_L0R_000000_20200328T032332_202003T032334_MTF_12196_7333.tif',
                               'baotou_CALVAL_L0R_000000_20200328T032332_202003T032334_MTF_12196_7333.tif')


In [ ]:
import mtf_estimator_algorithm
import osgeo
import numpy as np
from osgeo import ogr, osr, gdal

%matplotlib widget


# Declare inputs

In [ ]:
# Area of Interest polygon in WKT
Aoi_Wkt = '''
Polygon ((29.32538932146828614 -37.22900444938820641, 
83.82494438264737369 -52.0925194660734121, 
90.31298665183538787 -25.78645717463848186, 
25.66849276974415517 -9.27144048943269894, 
29.32538932146828614 -37.22900444938820641))
'''

# Raster to mask and crop using Aoi_Wkt
Image_File = 'baotou_CALVAL_L0R_000000_20200328T032332_202003T032334_MTF_12196_7333.tif'
# Band number. First is 1
Band_n = 1


# Read, mask and crop input file

In [ ]:
gdal_layer = gdal.Open(Image_File, gdal.GA_ReadOnly)
gt = list(gdal_layer.GetGeoTransform())
xsize = gdal_layer.RasterXSize
ysize = gdal_layer.RasterYSize
band = gdal_layer.GetRasterBand(Band_n)
raster_srs = osr.SpatialReference()
raster_srs.ImportFromWkt(gdal_layer.GetProjection())
vector_srs = raster_srs


# https://gdal.org/tutorials/osr_api_tut.html#crs-and-axis-order
if int(osgeo.__version__[0]) >= 3:
    # GDAL 3 changes axis order: https://github.com/OSGeo/gdal/issues/1546
    raster_srs.SetAxisMappingStrategy(osgeo.osr.OAMS_TRADITIONAL_GIS_ORDER)
    vector_srs.SetAxisMappingStrategy(osgeo.osr.OAMS_TRADITIONAL_GIS_ORDER)

if str(raster_srs) == '':
    coord_transform = None
    # self.console('WARNING: Raster with no CRS')
    gt[5] = -1*gt[5]
else:
    coord_transform = osr.CoordinateTransformation(vector_srs, raster_srs)


memlayer_drv = ogr.GetDriverByName('Memory')
memlayer_ds = memlayer_drv.CreateDataSource('')
memlayer = memlayer_ds.CreateLayer('aoi', raster_srs, geom_type=ogr.wkbPolygon)
memlayer.CreateField(ogr.FieldDefn('id', ogr.OFTInteger))
featureDefn = memlayer.GetLayerDefn()
memfeat = ogr.Feature(featureDefn)
geom = ogr.CreateGeometryFromWkt(Aoi_Wkt)
if not coord_transform is None:
    geom.Transform(coord_transform)

memfeat.SetGeometry(geom)
memlayer.CreateFeature(memfeat)

# Get extent in raster coords
e = np.array(memlayer.GetExtent()).copy()
e = np.reshape(e, [2, 2])
e = np.array(np.meshgrid(e[0], e[1]))
E = e.T.reshape(-1, 2)
m = np.reshape(np.array(gt).copy(), [2, 3])
A = m[:, 0]
m = m[:, 1:]
M = np.linalg.inv(m)
col_list, row_list = np.matmul(M, (E-A).T)
pxoffset = 5
col_min = int(np.max([np.floor(np.min(col_list)) - pxoffset, 1]))
col_max = int(np.min([np.ceil(np.max(col_list))+pxoffset, xsize-1]))
row_min = int(np.max([np.floor(np.min(row_list)) - pxoffset, 1]))
row_max = int(np.min([np.ceil(np.max(row_list))+pxoffset, ysize-1]))
sub_gt = gt
sub_gt[0] = gt[0] + gt[1]*col_min + gt[2]*row_min
sub_gt[3] = gt[3] + gt[4]*col_min + gt[5]*row_min
sub_xsize = int(col_max-col_min)
sub_ysize = int(row_max-row_min)

memraster_drv = gdal.GetDriverByName('MEM')
memraster = memraster_drv.Create('', sub_xsize, sub_ysize, 1, band.DataType)

memraster.SetProjection(gdal_layer.GetProjection())
memraster.SetGeoTransform(sub_gt)
memband = memraster.GetRasterBand(1)
memband.WriteArray(np.zeros([sub_ysize, sub_xsize]))
gdal.RasterizeLayer(memraster, [1], memlayer, burn_values=[1])
mask = memband.ReadAsArray(0, 0, sub_xsize, sub_ysize)
memband.WriteArray(mask*band.ReadAsArray(col_min, row_min, sub_xsize, sub_ysize))
mask = None


# Run algorithm

In [ ]:
mtf_estimator_algorithm.Mtf(memraster)
